In [277]:
import pandas as pd 
import numpy as np 
import requests
from pandas.io.json import json_normalize

In [278]:
from geopy.geocoders import Nominatim 
import requests 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 

### Foursquare API credentials

In [279]:
CLIENT_ID = 'WQAEIEJSBV3TBDMRTRKPAQWIAGVMS4MOTDRILYGOME3YH31W' # your Foursquare ID
CLIENT_SECRET = 'A0SHX4J5Z1Q4U52QS25QXTB352VZHTTAZVUW1NLMKYD3M2R5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
ACCESS_TOKEN = 'N1UEH5XOMCCMMCXCUH3XTQJIN1T3RUCQUVCKMBGE2JHBM0Z1'
LIMIT = 100 # A default Foursquare API limit value
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
print('Access Token:', ACCESS_TOKEN)

Your credentails:
CLIENT_ID: WQAEIEJSBV3TBDMRTRKPAQWIAGVMS4MOTDRILYGOME3YH31W
CLIENT_SECRET:A0SHX4J5Z1Q4U52QS25QXTB352VZHTTAZVUW1NLMKYD3M2R5
Access Token: N1UEH5XOMCCMMCXCUH3XTQJIN1T3RUCQUVCKMBGE2JHBM0Z1


# Chennai

In [280]:
address = 'Chennai, TN'
geolocator = Nominatim(user_agent='Hos_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The coordinates of {} is {}, {}'.format(address, latitude, longitude))

The coordinates of Chennai, TN is 13.0836939, 80.270186


In [281]:
chennai_latitude = latitude
chennai_longitude = longitude

In [282]:
radius = 30000

url = 'https://api.foursquare.com/v2/venues/search?categoryId=4bf58dd8d48988d196941735&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, VERSION, chennai_latitude, chennai_longitude, radius, LIMIT)

url

'https://api.foursquare.com/v2/venues/search?categoryId=4bf58dd8d48988d196941735&client_id=WQAEIEJSBV3TBDMRTRKPAQWIAGVMS4MOTDRILYGOME3YH31W&client_secret=A0SHX4J5Z1Q4U52QS25QXTB352VZHTTAZVUW1NLMKYD3M2R5&v=20180605&ll=13.0836939,80.270186&radius=30000&limit=100'

In [283]:
results = requests.get(url).json()

In [284]:
results['response']['venues'][0]

{'id': '51ecf8bf498e595e22d17606',
 'name': 'Rajiv Gandhi Government General Hospital',
 'location': {'address': 'Opp.Chennai Central',
  'crossStreet': 'Poonamalle High Road',
  'lat': 13.081822221837289,
  'lng': 80.2767191306967,
  'labeledLatLngs': [{'label': 'display',
    'lat': 13.081822221837289,
    'lng': 80.2767191306967}],
  'distance': 738,
  'postalCode': '600003',
  'cc': 'IN',
  'city': 'Chennai',
  'state': 'Tamil Nadu',
  'country': 'India',
  'formattedAddress': ['Opp.Chennai Central (Poonamalle High Road)',
   'Chennai 600003',
   'Tamil Nadu',
   'India']},
 'categories': [{'id': '4bf58dd8d48988d196941735',
   'name': 'Hospital',
   'pluralName': 'Hospitals',
   'shortName': 'Hospital',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/medical_',
    'suffix': '.png'},
   'primary': True}],
 'referralId': 'v-1621099354',
 'hasPerk': False}

## Creating DataFrame for Chennai Hospitals

In [285]:
chennai_hospitals_df = pd.DataFrame(columns=['Hospital name', 'category', 'Latitude', 'Longitude'])

In [286]:
chennai_hospitals_venues = results['response']['venues']

In [287]:
chennai_hospitals = json_normalize(chennai_hospitals_venues)


<ipython-input-287-a344204aa7ba>:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  chennai_hospitals = json_normalize(chennai_hospitals_venues)


In [288]:
chennai_hospitals.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood
0,51ecf8bf498e595e22d17606,Rajiv Gandhi Government General Hospital,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1621099354,False,Opp.Chennai Central,Poonamalle High Road,13.081822,80.276719,"[{'label': 'display', 'lat': 13.08182222183728...",738,600003,IN,Chennai,Tamil Nadu,India,"[Opp.Chennai Central (Poonamalle High Road), C...",NaN
1,4eb65ab17ee5024d7f68bb5a,Miot Hospital,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1621099354,False,Ramapuram,"4/112, Mount Poonamalle Street.",13.021841,80.185831,"[{'label': 'display', 'lat': 13.02184112757657...",11449,600089,IN,Chennai,Tamil Nadu,India,"[Ramapuram (4/112, Mount Poonamalle Street.), ...",Manapakkam
2,5de3826d5bdd9e000829c942,Aravind Eye Hospital - Chennai,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1621099354,False,Poonamalle High Road,Opposite to Saveetha Dental College,13.053257,80.124491,"[{'label': 'display', 'lat': 13.053257, 'lng':...",16157,600077,IN,Chennai,Tamil Nadu,India,[Poonamalle High Road (Opposite to Saveetha De...,NaN
3,5c021f305d891b002c927e9f,Apollo Proton Cancer Centre,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1621099354,False,NaN,NaN,12.980291,80.250884,"[{'label': 'display', 'lat': 12.980291, 'lng':...",11699,600041,IN,Chennai,Tamil Nadu,India,"[Chennai 600041, Tamil Nadu, India]",NaN
4,4cccf3d8ba79a1cdf6c83ecb,Madras Medical Mission (MMM Hospital),"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1621099354,False,"4-A, Dr. J. Jayalalitha Nagar,",Mogappair,13.085986,80.187398,"[{'label': 'display', 'lat': 13.08598577011605...",8980,600037,IN,Chennai,Tamil Nadu,India,"[4-A, Dr. J. Jayalalitha Nagar, (Mogappair), C...",NaN


In [289]:
chennai_hospitals = chennai_hospitals[['name', 'categories', 'location.lat', 'location.lng', 'location.city']]

In [290]:
chennai_hospitals.head()

,name,categories,location.lat,location.lng,location.city
0,Rajiv Gandhi Government General Hospital,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",13.081822,80.276719,Chennai
1,Miot Hospital,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",13.021841,80.185831,Chennai
2,Aravind Eye Hospital - Chennai,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",13.053257,80.124491,Chennai
3,Apollo Proton Cancer Centre,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",12.980291,80.250884,Chennai
4,Madras Medical Mission (MMM Hospital),"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",13.085986,80.187398,Chennai


In [291]:
chennai_hospitals['categories'][0]

[{'id': '4bf58dd8d48988d196941735',
  'name': 'Hospital',
  'pluralName': 'Hospitals',
  'shortName': 'Hospital',
  'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/medical_',
   'suffix': '.png'},
  'primary': True}]

In [292]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [293]:
chennai_hospitals['categories'] = chennai_hospitals.apply(get_category_type, axis=1)

In [294]:
print(chennai_hospitals.shape)
chennai_hospitals.head()

(50, 5)


,name,categories,location.lat,location.lng,location.city
0,Rajiv Gandhi Government General Hospital,Hospital,13.081822,80.276719,Chennai
1,Miot Hospital,Hospital,13.021841,80.185831,Chennai
2,Aravind Eye Hospital - Chennai,Hospital,13.053257,80.124491,Chennai
3,Apollo Proton Cancer Centre,Hospital,12.980291,80.250884,Chennai
4,Madras Medical Mission (MMM Hospital),Hospital,13.085986,80.187398,Chennai


In [295]:
chennai_hospitals.rename(columns={'name':'Name', 'categories':'Category', 'location.lat':'Latitude', 'location.lng':'Longitude', 'location.city':'City Name'}, inplace=True)

### From the Dataframe Removing the place which doen't treat COVID

In [296]:
### if these words are present that is not full fledge Hospital
to_remove_row = ['Clinic', 'Health', "Children", 'Child', 'Fertility', 'Eye', 'doctor', 'Doctor', 'Care', 'Diabetes', 'Home', 'BRAIN', 'Brain', 'Centre', 'centre', 'Center', 'Maternity', 'Surgeries', 'Cradle', 'cardle', 'Speciality', 'Oncology']
chennai_hospitals = chennai_hospitals[~chennai_hospitals.Name.str.contains('|'.join(to_remove_row))].reset_index(drop=True)

In [297]:
chennai_hospitals['City Name'].replace(to_replace=np.NaN, value='Chennai', inplace=True)
chennai_hospitals['City Name'].replace(to_replace="adambakkam, chennai", value='Chennai', inplace=True)
chennai_hospitals['City Name'].replace(to_replace='Pallāvaram', value='Chennai', inplace=True)

## Final List of  chennai Hospitals DataFrame

In [298]:
chennai_hospitals

,Name,Category,Latitude,Longitude,City Name
0,Rajiv Gandhi Government General Hospital,Hospital,13.081822,80.276719,Chennai
1,Miot Hospital,Hospital,13.021841,80.185831,Chennai
2,Madras Medical Mission (MMM Hospital),Hospital,13.085986,80.187398,Chennai
3,MIOT International,Hospital,13.022287,80.186130,Chennai
4,Apollo Hospitals,Hospital,13.062761,80.251790,Chennai
5,Saveetha Medical College,College Science Building,13.023830,80.016896,Chennai
6,JK Hospital,Hospital,13.034600,80.164136,Chennai
7,St Anthonys Hospital,Hospital,13.155233,80.228571,Chennai
8,Cantonment General Hospital,Hospital,12.970381,80.148503,Chennai
9,Surya hospital,Hospital,13.057873,80.264103,Chennai


### Chennai Hospital Location

In [299]:
chennai_hospitals_map = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, long, name in zip(chennai_hospitals['Latitude'], chennai_hospitals['Longitude'], chennai_hospitals['Name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        location=[lat, long],
        radius=6,
        popup=label,
        color='Blue',
        fill=True,
        fill_color='#90EE90',
        fill_opacity=0.7,
        parse_html=False
    ).add_to(chennai_hospitals_map)

chennai_hospitals_map

# Banglore

In [300]:
address = 'Bengaluru, KA'
geolocator = Nominatim(user_agent='Hos_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The coordinates of {} is {}, {}'.format(address, latitude, longitude))

The coordinates of Bengaluru, KA is 12.9791198, 77.5912997


### Bengaluru and banglore both represent same place

In [301]:
banglore_latitude = latitude
banglore_longitude = longitude

In [302]:
radius = 20000

url = 'https://api.foursquare.com/v2/venues/search?categoryId=4bf58dd8d48988d196941735&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, VERSION, banglore_latitude, banglore_longitude, radius, LIMIT)

url

'https://api.foursquare.com/v2/venues/search?categoryId=4bf58dd8d48988d196941735&client_id=WQAEIEJSBV3TBDMRTRKPAQWIAGVMS4MOTDRILYGOME3YH31W&client_secret=A0SHX4J5Z1Q4U52QS25QXTB352VZHTTAZVUW1NLMKYD3M2R5&v=20180605&ll=12.9791198,77.5912997&radius=20000&limit=100'

In [303]:
results = requests.get(url).json()

In [304]:
results['response']['venues'][0]

{'id': '4ec6613082311ff83c7fd7b6',
 'name': 'Sparsh Hospital',
 'location': {'lat': 12.983104839777111,
  'lng': 77.59501467327372,
  'labeledLatLngs': [{'label': 'display',
    'lat': 12.983104839777111,
    'lng': 77.59501467327372}],
  'distance': 599,
  'cc': 'IN',
  'country': 'India',
  'formattedAddress': ['India']},
 'categories': [{'id': '4bf58dd8d48988d196941735',
   'name': 'Hospital',
   'pluralName': 'Hospitals',
   'shortName': 'Hospital',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/medical_',
    'suffix': '.png'},
   'primary': True}],
 'referralId': 'v-1621099358',
 'hasPerk': False}

In [305]:
banglore_hospitals_df = pd.DataFrame(columns=['Hospital name', 'category', 'Latitude', 'Longitude'])
banglore_hospitals_venues = results['response']['venues']
banglore_hospitals = json_normalize(banglore_hospitals_venues)

<ipython-input-305-30cb1193f914>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  banglore_hospitals = json_normalize(banglore_hospitals_venues)


In [306]:
banglore_hospitals.head()

,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.country,location.formattedAddress,location.address,location.crossStreet,location.postalCode,location.city,location.state,venuePage.id,location.neighborhood
0,4ec6613082311ff83c7fd7b6,Sparsh Hospital,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1621099358,False,12.983105,77.595015,"[{'label': 'display', 'lat': 12.98310483977711...",599,IN,India,[India],NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5b712c3a35811b002c38e25c,Manipal Hospital - Jayanagar,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1621099358,False,12.917354,77.591888,"[{'label': 'display', 'lat': 12.91735366593119...",6876,IN,India,"[5/1, 45th Cross, Opp Bangalore Central Mall, ...","5/1, 45th Cross, Opp Bangalore Central Mall, 9...","9th Block, Jayanagar",560069,Bangalore,Karnātaka,NaN,NaN
2,57963667cd10f4e250a41c84,Rainbow Children's Hospital,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1621099358,False,12.903372,77.600950,"[{'label': 'display', 'lat': 12.903372, 'lng':...",8496,IN,India,[India],NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,517e18e7e4b04f0bfad64f9d,HBS HOSPITAL,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1621099358,False,12.989273,77.603705,"[{'label': 'display', 'lat': 12.98927312666556...",1757,IN,India,[India],NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,58747efb761b1a0fb6ccaae2,"Manipal Hospital, Whitefield","[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1621099358,False,12.988572,77.728825,"[{'label': 'display', 'lat': 12.988572, 'lng':...",14954,IN,India,[India],NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [307]:
banglore_hospitals = banglore_hospitals[['name', 'categories', 'location.lat', 'location.lng', 'location.city']]
banglore_hospitals.head()

,name,categories,location.lat,location.lng,location.city
0,Sparsh Hospital,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",12.983105,77.595015,NaN
1,Manipal Hospital - Jayanagar,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",12.917354,77.591888,Bangalore
2,Rainbow Children's Hospital,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",12.903372,77.600950,NaN
3,HBS HOSPITAL,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",12.989273,77.603705,NaN
4,"Manipal Hospital, Whitefield","[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",12.988572,77.728825,NaN


In [308]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [309]:
banglore_hospitals['categories'] = banglore_hospitals.apply(get_category_type, axis=1)
print(banglore_hospitals.shape)
banglore_hospitals.head()

(50, 5)


,name,categories,location.lat,location.lng,location.city
0,Sparsh Hospital,Hospital,12.983105,77.595015,NaN
1,Manipal Hospital - Jayanagar,Hospital,12.917354,77.591888,Bangalore
2,Rainbow Children's Hospital,Hospital,12.903372,77.600950,NaN
3,HBS HOSPITAL,Hospital,12.989273,77.603705,NaN
4,"Manipal Hospital, Whitefield",Hospital,12.988572,77.728825,NaN


In [310]:
banglore_hospitals.rename(columns={'name':'Name', 'categories':'Category', 'location.lat':'Latitude', 'location.lng':'Longitude', 'location.city':'City Name'}, inplace=True)

In [311]:
banglore_hospitals = banglore_hospitals[~banglore_hospitals.Name.str.contains('|'.join(to_remove_row))].reset_index(drop=True)

In [312]:
banglore_hospitals['City Name'].replace(to_replace=np.NaN, value='Banglore', inplace=True)

In [313]:
print(banglore_hospitals.shape)
banglore_hospitals

(35, 5)


,Name,Category,Latitude,Longitude,City Name
0,Sparsh Hospital,Hospital,12.983105,77.595015,Banglore
1,Manipal Hospital - Jayanagar,Hospital,12.917354,77.591888,Bangalore
2,HBS HOSPITAL,Hospital,12.989273,77.603705,Banglore
3,"Manipal Hospital, Whitefield",Hospital,12.988572,77.728825,Banglore
4,Sagar Chandramma Hospital,Hospital,12.948401,77.574827,Bangalore
5,Chinmaya Mission Hospital,Hospital,12.977870,77.645956,Bangalore
6,Sakra World Hospital,Hospital,12.931862,77.685472,Bangalore
7,M.S Ramaiah Memorial Hospital,Hospital,13.028492,77.569780,Bangalore
8,Manipal Hospital,Hospital,12.959238,77.649208,Bangalore
9,Apollo Hospital,Hospital,12.896250,77.598653,Bangalore


### Banglore Hospital location Map

In [314]:
banglore_hospitals_map = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, long, name in zip(banglore_hospitals['Latitude'], banglore_hospitals['Longitude'], banglore_hospitals['Name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        location=[lat, long],
        radius=6,
        popup=label,
        color='Blue',
        fill=True,
        fill_color='#90EE90',
        fill_opacity=0.7,
        parse_html=False
    ).add_to(banglore_hospitals_map)

banglore_hospitals_map

# Mumbai

In [315]:
address = 'Mumbai'
geolocator = Nominatim(user_agent='Hos_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The coordinates of {} is {}, {}'.format(address, latitude, longitude))

The coordinates of Mumbai is 19.0759899, 72.8773928


In [316]:
mumbai_latitude = latitude
mumbai_longitude = longitude

In [317]:
radius = 20000

url = 'https://api.foursquare.com/v2/venues/search?categoryId=4bf58dd8d48988d196941735&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, VERSION, mumbai_latitude, mumbai_longitude, radius, LIMIT)

url

'https://api.foursquare.com/v2/venues/search?categoryId=4bf58dd8d48988d196941735&client_id=WQAEIEJSBV3TBDMRTRKPAQWIAGVMS4MOTDRILYGOME3YH31W&client_secret=A0SHX4J5Z1Q4U52QS25QXTB352VZHTTAZVUW1NLMKYD3M2R5&v=20180605&ll=19.0759899,72.8773928&radius=20000&limit=100'

In [318]:
results = requests.get(url).json()

In [319]:
results['response']['venues'][0]

{'id': '53304806498e643c06ecae0d',
 'name': 'kohinoor hospital',
 'location': {'address': 'Kohinoor City',
  'lat': 19.07671707918599,
  'lng': 72.88627483472581,
  'labeledLatLngs': [{'label': 'display',
    'lat': 19.07671707918599,
    'lng': 72.88627483472581}],
  'distance': 937,
  'cc': 'IN',
  'city': 'Mumbai',
  'state': 'Mahārāshtra',
  'country': 'India',
  'formattedAddress': ['Kohinoor City', 'Mumbai', 'Mahārāshtra', 'India']},
 'categories': [{'id': '4bf58dd8d48988d196941735',
   'name': 'Hospital',
   'pluralName': 'Hospitals',
   'shortName': 'Hospital',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/medical_',
    'suffix': '.png'},
   'primary': True}],
 'referralId': 'v-1621099361',
 'hasPerk': False}

In [320]:
mumbai_hospitals_df = pd.DataFrame(columns=['Hospital name', 'category', 'Latitude', 'Longitude'])
mumbai_hospitals_venues = results['response']['venues']
mumbai_hospitals = json_normalize(mumbai_hospitals_venues)

<ipython-input-320-46e9b5d74eb4>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  mumbai_hospitals = json_normalize(mumbai_hospitals_venues)


In [321]:
mumbai_hospitals = mumbai_hospitals[['name', 'categories', 'location.lat', 'location.lng', 'location.city']]
mumbai_hospitals.head()

,name,categories,location.lat,location.lng,location.city
0,kohinoor hospital,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",19.076717,72.886275,Mumbai
1,BKC Covid centre,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",19.067807,72.864099,Mumbai
2,Nanavati Hospital,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",19.095887,72.839919,Mumbai
3,Saifee Hospital,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",18.952554,72.817934,Mumbai
4,Dr. L H Hiranandani Hospital,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",19.120386,72.916633,Mumbai


In [322]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [323]:
mumbai_hospitals['categories'] = mumbai_hospitals.apply(get_category_type, axis=1)
print(mumbai_hospitals.shape)
mumbai_hospitals.head()

(50, 5)


,name,categories,location.lat,location.lng,location.city
0,kohinoor hospital,Hospital,19.076717,72.886275,Mumbai
1,BKC Covid centre,Hospital,19.067807,72.864099,Mumbai
2,Nanavati Hospital,Hospital,19.095887,72.839919,Mumbai
3,Saifee Hospital,Hospital,18.952554,72.817934,Mumbai
4,Dr. L H Hiranandani Hospital,Hospital,19.120386,72.916633,Mumbai


In [324]:
mumbai_hospitals.rename(columns={'name':'Name', 'categories':'Category', 'location.lat':'Latitude', 'location.lng':'Longitude', 'location.city':'City Name'}, inplace=True)

In [325]:
mumbai_hospitals = mumbai_hospitals[~mumbai_hospitals.Name.str.contains('|'.join(to_remove_row))].reset_index(drop=True)

mumbai_hospitals['City Name'].replace(to_replace=np.NaN, value='mumbai', inplace=True)

mumbai_hospitals.drop(index=10, inplace=True)
print(mumbai_hospitals.shape)
mumbai_hospitals.reset_index(drop=True)

(38, 5)


,Name,Category,Latitude,Longitude,City Name
0,kohinoor hospital,Hospital,19.076717,72.886275,Mumbai
1,Nanavati Hospital,Hospital,19.095887,72.839919,Mumbai
2,Saifee Hospital,Hospital,18.952554,72.817934,Mumbai
3,Dr. L H Hiranandani Hospital,Hospital,19.120386,72.916633,Mumbai
4,Sion Hospital (LTMGH),Hospital,19.035965,72.859622,Sion West
5,Bhatia General Hospital,Hospital,18.965575,72.813265,Mumbai
6,Prince Aly Khan Hospital,Hospital,18.970985,72.836377,Mumbai
7,Fortis Hospital,Hospital,19.162155,72.942190,Mumbai
8,Bhabha Hospital,Hospital,19.057727,72.833855,Mumbai
9,Powai Polyclinic,Hospital,19.124175,72.917037,mumbai


## Mumbai hospital location

In [326]:
mumbai_hospitals_map = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, long, name in zip(mumbai_hospitals['Latitude'], mumbai_hospitals['Longitude'], mumbai_hospitals['Name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        location=[lat, long],
        radius=6,
        popup=label,
        color='Blue',
        fill=True,
        fill_color='#90EE90',
        fill_opacity=0.7,
        parse_html=False
    ).add_to(mumbai_hospitals_map)

mumbai_hospitals_map

# New Delhi

In [327]:
address = 'New Delhi'
geolocator = Nominatim(user_agent='Hos_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The coordinates of {} is {}, {}'.format(address, latitude, longitude))

The coordinates of New Delhi is 28.6138954, 77.2090057


In [328]:
newdelhi_latitude = latitude
newdelhi_longitude = longitude

In [329]:
radius = 15000

url = 'https://api.foursquare.com/v2/venues/search?categoryId=4bf58dd8d48988d196941735&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, VERSION, newdelhi_latitude, newdelhi_longitude, radius, LIMIT)

url

'https://api.foursquare.com/v2/venues/search?categoryId=4bf58dd8d48988d196941735&client_id=WQAEIEJSBV3TBDMRTRKPAQWIAGVMS4MOTDRILYGOME3YH31W&client_secret=A0SHX4J5Z1Q4U52QS25QXTB352VZHTTAZVUW1NLMKYD3M2R5&v=20180605&ll=28.6138954,77.2090057&radius=15000&limit=100'

In [330]:
results = requests.get(url).json()

In [331]:
results['response']['venues'][0]

{'id': '4e1f1430cc3f012b9756fdfd',
 'name': 'Dr. Ram Manohar Lohia Hospital',
 'location': {'address': 'Ram Manohar Lohia Hospital',
  'lat': 28.624654222133696,
  'lng': 77.20060862393855,
  'labeledLatLngs': [{'label': 'display',
    'lat': 28.624654222133696,
    'lng': 77.20060862393855}],
  'distance': 1451,
  'cc': 'IN',
  'city': 'New Delhi',
  'state': 'Delhi',
  'country': 'India',
  'formattedAddress': ['Ram Manohar Lohia Hospital',
   'New Delhi',
   'Delhi',
   'India']},
 'categories': [{'id': '4bf58dd8d48988d196941735',
   'name': 'Hospital',
   'pluralName': 'Hospitals',
   'shortName': 'Hospital',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/medical_',
    'suffix': '.png'},
   'primary': True}],
 'referralId': 'v-1621099364',
 'hasPerk': False}

In [332]:
newdelhi_hospitals_df = pd.DataFrame(columns=['Hospital name', 'category', 'Latitude', 'Longitude'])
newdelhi_hospitals_venues = results['response']['venues']
newdelhi_hospitals = json_normalize(newdelhi_hospitals_venues)

<ipython-input-332-5317f438d92a>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  newdelhi_hospitals = json_normalize(newdelhi_hospitals_venues)


In [333]:
newdelhi_hospitals = newdelhi_hospitals[['name', 'categories', 'location.lat', 'location.lng', 'location.city']]
newdelhi_hospitals.head()

,name,categories,location.lat,location.lng,location.city
0,Dr. Ram Manohar Lohia Hospital,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",28.624654,77.200609,New Delhi
1,Sir Ganga Ram Hospital | सर गंगा राम अस्पताल (...,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",28.638180,77.189244,Delhi
2,Apollo Hospital,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",28.579586,77.334841,Noida
3,Apollo Hospital,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",28.541229,77.283508,New Delhi
4,AIIMS,"[{'id': '4bf58dd8d48988d1b3941735', 'name': 'M...",28.567922,77.209255,New Delhi


In [334]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [335]:
newdelhi_hospitals['categories'] = newdelhi_hospitals.apply(get_category_type, axis=1)
print(newdelhi_hospitals.shape)
newdelhi_hospitals.head()

(50, 5)


,name,categories,location.lat,location.lng,location.city
0,Dr. Ram Manohar Lohia Hospital,Hospital,28.624654,77.200609,New Delhi
1,Sir Ganga Ram Hospital | सर गंगा राम अस्पताल (...,Hospital,28.638180,77.189244,Delhi
2,Apollo Hospital,Hospital,28.579586,77.334841,Noida
3,Apollo Hospital,Hospital,28.541229,77.283508,New Delhi
4,AIIMS,Medical School,28.567922,77.209255,New Delhi


In [336]:
newdelhi_hospitals.rename(columns={'name':'Name', 'categories':'Category', 'location.lat':'Latitude', 'location.lng':'Longitude', 'location.city':'City Name'}, inplace=True)

In [337]:
newdelhi_hospitals = newdelhi_hospitals[~newdelhi_hospitals.Name.str.contains('|'.join(to_remove_row))].reset_index(drop=True)

newdelhi_hospitals['City Name'].replace(to_replace=np.NaN, value='newdelhi', inplace=True)

print(newdelhi_hospitals.shape)
newdelhi_hospitals.reset_index(drop=True)

(43, 5)


,Name,Category,Latitude,Longitude,City Name
0,Dr. Ram Manohar Lohia Hospital,Hospital,28.624654,77.200609,New Delhi
1,Sir Ganga Ram Hospital | सर गंगा राम अस्पताल (...,Hospital,28.638180,77.189244,Delhi
2,Apollo Hospital,Hospital,28.579586,77.334841,Noida
3,Apollo Hospital,Hospital,28.541229,77.283508,New Delhi
4,AIIMS,Medical School,28.567922,77.209255,New Delhi
5,Fortis hospital,Hospital,28.519234,77.160670,New Delhi
6,ESI Hospital,Hospital,28.654616,77.127244,newdelhi
7,ADIVA,Hospital,28.558334,77.206657,New Delhi
8,Metro Heart Institute,Hospital,28.564741,77.243335,New Delhi
9,"Jaipur Golden Hospital, Rohini (Delhi)",Hospital,28.699115,77.108287,newdelhi


## Newdelhi hospital location

In [338]:
newdelhi_hospitals_map = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, long, name in zip(newdelhi_hospitals['Latitude'], newdelhi_hospitals['Longitude'], newdelhi_hospitals['Name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        location=[lat, long],
        radius=6,
        popup=label,
        color='Blue',
        fill=True,
        fill_color='#90EE90',
        fill_opacity=0.7,
        parse_html=False
    ).add_to(newdelhi_hospitals_map)

newdelhi_hospitals_map

# Calcutta

In [339]:
address = 'Calcutta'
geolocator = Nominatim(user_agent='Hos_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The coordinates of {} is {}, {}'.format(address, latitude, longitude))

The coordinates of Calcutta is 22.5726723, 88.3638815


In [340]:
calcutta_latitude = latitude
calcutta_longitude = longitude

In [341]:
radius = 15000

url = 'https://api.foursquare.com/v2/venues/search?categoryId=4bf58dd8d48988d196941735&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, VERSION, calcutta_latitude, calcutta_longitude, radius, LIMIT)

url

'https://api.foursquare.com/v2/venues/search?categoryId=4bf58dd8d48988d196941735&client_id=WQAEIEJSBV3TBDMRTRKPAQWIAGVMS4MOTDRILYGOME3YH31W&client_secret=A0SHX4J5Z1Q4U52QS25QXTB352VZHTTAZVUW1NLMKYD3M2R5&v=20180605&ll=22.5726723,88.3638815&radius=15000&limit=100'

In [342]:
results = requests.get(url).json()

In [343]:
results['response']['venues'][0]

{'id': '55946770498ee799d001de1f',
 'name': 'Narayana Multispeciality Hospital',
 'location': {'address': 'Andul Rd',
  'lat': 22.567414749067407,
  'lng': 88.27152013778687,
  'labeledLatLngs': [{'label': 'display',
    'lat': 22.567414749067407,
    'lng': 88.27152013778687}],
  'distance': 9512,
  'postalCode': '711109',
  'cc': 'IN',
  'city': 'Hāora',
  'state': 'West Bengal',
  'country': 'India',
  'formattedAddress': ['Andul Rd', 'Hāora 711109', 'West Bengal', 'India']},
 'categories': [{'id': '4bf58dd8d48988d196941735',
   'name': 'Hospital',
   'pluralName': 'Hospitals',
   'shortName': 'Hospital',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/medical_',
    'suffix': '.png'},
   'primary': True}],
 'referralId': 'v-1621099367',
 'hasPerk': False}

In [344]:
calcutta_hospitals_df = pd.DataFrame(columns=['Hospital name', 'category', 'Latitude', 'Longitude'])
calcutta_hospitals_venues = results['response']['venues']
calcutta_hospitals = json_normalize(calcutta_hospitals_venues)

<ipython-input-344-9476d4663064>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  calcutta_hospitals = json_normalize(calcutta_hospitals_venues)


In [345]:
calcutta_hospitals = calcutta_hospitals[['name', 'categories', 'location.lat', 'location.lng', 'location.city']]
calcutta_hospitals.head()

,name,categories,location.lat,location.lng,location.city
0,Narayana Multispeciality Hospital,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",22.567415,88.271520,Hāora
1,Tata Medical Center,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",22.576707,88.480239,Kolkata
2,Nightingale Hospital,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",22.545964,88.351471,Kolkata
3,Medicine Department,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",22.493942,88.373358,Kolkata
4,Rajarhat Gopalpur Municipal Hospital,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",22.616145,88.426854,Kolkata


In [346]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [347]:
calcutta_hospitals['categories'] = calcutta_hospitals.apply(get_category_type, axis=1)
print(calcutta_hospitals.shape)
calcutta_hospitals.head()

(35, 5)


,name,categories,location.lat,location.lng,location.city
0,Narayana Multispeciality Hospital,Hospital,22.567415,88.271520,Hāora
1,Tata Medical Center,Hospital,22.576707,88.480239,Kolkata
2,Nightingale Hospital,Hospital,22.545964,88.351471,Kolkata
3,Medicine Department,Hospital,22.493942,88.373358,Kolkata
4,Rajarhat Gopalpur Municipal Hospital,Hospital,22.616145,88.426854,Kolkata


In [348]:
calcutta_hospitals.rename(columns={'name':'Name', 'categories':'Category', 'location.lat':'Latitude', 'location.lng':'Longitude', 'location.city':'City Name'}, inplace=True)

In [349]:
calcutta_hospitals = calcutta_hospitals[~calcutta_hospitals.Name.str.contains('|'.join(to_remove_row))].reset_index(drop=True)

calcutta_hospitals['City Name'].replace(to_replace=np.NaN, value='calcutta', inplace=True)

calcutta_hospitals.drop(index=[10,21], inplace=True)
print(calcutta_hospitals.shape)
calcutta_hospitals.reset_index(drop=True)

(24, 5)


,Name,Category,Latitude,Longitude,City Name
0,Narayana Multispeciality Hospital,Hospital,22.567415,88.271520,Hāora
1,Nightingale Hospital,Hospital,22.545964,88.351471,Kolkata
2,Medicine Department,Hospital,22.493942,88.373358,Kolkata
3,Rajarhat Gopalpur Municipal Hospital,Hospital,22.616145,88.426854,Kolkata
4,Ramakrishna Mission Seva Prathishthan,Hospital,22.523925,88.351901,Kolkata
5,Surgery Department,Hospital,22.494022,88.373380,Kolkata
6,South Dum Dum Municipal Hospital,Hospital,22.622900,88.413460,Kolkata
7,Block 3,Hospital,22.495070,88.372925,Kolkata
8,B.R.S.H Hospital,Hospital,22.573163,88.367367,Kolkata
9,Central Reception,Hospital,22.493843,88.373594,Kolkata


## Calcutta hospital location

In [350]:
calcutta_hospitals_map = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, long, name in zip(calcutta_hospitals['Latitude'], calcutta_hospitals['Longitude'], calcutta_hospitals['Name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        location=[lat, long],
        radius=6,
        popup=label,
        color='Blue',
        fill=True,
        fill_color='#90EE90',
        fill_opacity=0.7,
        parse_html=False
    ).add_to(calcutta_hospitals_map)

calcutta_hospitals_map